# Artifact Table 1-A for Phase Moducation Side Channel Analysis and JTAG Attacks

This is part of a series of artifacts. Note these notebooks are designed to run on Google Colab so have been simplified from those available in the [full repository](https://github.com/colinoflynn/phase-modulation-sca).

To run these notebooks, simply run from top to bottom.

**Section of Paper Reproduced**: These will reproduce CPA results in Table 1.

**WARNING**: Some steps, such as the extraction, can be very slow. Try not to forget you started the process as CoLab will kill unattended notebooks eventually, and you'll have to start again.

In [ ]:
!pip install scared

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.6 MB/s eta 0:00:00


In [ ]:
!wget https://www.dropbox.com/scl/fi/tlomuvppihg4y1z2x1gfr/baseline_traces_cpa.7z?rlkey=hdn5xpepjedid96o7y7drkjlf&dl=1

--2024-07-26 17:32:51--  https://www.dropbox.com/scl/fi/tlomuvppihg4y1z2x1gfr/baseline_traces_cpa.7z?rlkey=hdn5xpepjedid96o7y7drkjlf
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucde740c7dfb873c7890ad8fd5e0.dl.dropboxusercontent.com/cd/0/inline/CXeiRUXeeNTx1d_8ri-b4e7h90V-V06KwM1fKy-6k3OnH2IJAo5W04MCsZl2Z3zp_s9e9ut9T3ZKULl3ecyMrabAZFQ5mMDpuCrRl4jWpfMo2t65vM-ivVSG8J5TDqjfEVq9YUSMH6442oaYx0zmG1bC/file# [following]
--2024-07-26 17:32:52--  https://ucde740c7dfb873c7890ad8fd5e0.dl.dropboxusercontent.com/cd/0/inline/CXeiRUXeeNTx1d_8ri-b4e7h90V-V06KwM1fKy-6k3OnH2IJAo5W04MCsZl2Z3zp_s9e9ut9T3ZKULl3ecyMrabAZFQ5mMDpuCrRl4jWpfMo2t65vM-ivVSG8J5TDqjfEVq9YUSMH6442oaYx0zmG1bC/file
Resolving ucde740c7dfb873c7890ad8fd5e0.dl.dropboxusercontent.com (ucde740c7dfb873c7890ad8fd5e0.dl.dropboxusercontent.com)... 162.125.3.

In [ ]:
!7z x baseline_traces_cpa.7z?rlkey=hdn5xpepjedid96o7y7drkjlf


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 4747588370 bytes (4528 MiB)

Extracting archive: baseline_traces_cpa.7z?rlkey=hdn5xpepjedid96o7y7drkjlf
--
Path = baseline_traces_cpa.7z?rlkey=hdn5xpepjedid96o7y7drkjlf
Type = 7z
Physical Size = 4747588370
Headers Size = 394
Method = LZMA2:22
Solid = +
Blocks = 6

  0%      0% 1 - baseline_traces_cpa/baseline120mhz_async_120msps_cpa.ets                                                                   1% 1 - baseline_traces_cpa/baseline120mhz_async_120msps_cpa.ets                                                      

In [ ]:
!rm -rf  baseline_traces_cpa.7z?rlkey=hdn5xpepjedid96o7y7drkjlf

In [ ]:
import scared
import estraces
import numpy as np

In [ ]:
knownkey = [43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60]

In [ ]:
def find_success(att, knownkey):
    recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')
    print(recovered_masterkey == knownkey)

    for i in range(0, np.shape(att.convergence_traces)[2]):
        if np.all(np.argmax(att.convergence_traces[:,:,i], axis=0) == knownkey):
            return (i+1)*att.convergence_step
    return -1

In [ ]:
import scared
from scipy import signal


def do_cpa(filename, step=25, window=None, hpf=None, lpf=None):
    ths = estraces.read_ths_from_ets_file(filename)
    print(ths)

    selection_function = scared.aes.selection_functions.encrypt.FirstSubBytes()

    att = scared.CPAAttack(selection_function=selection_function,
                           model=scared.HammingWeight(),
                           discriminant=scared.maxabs,
                           convergence_step=step)

    processes = []

    # Simple high pass filter (HPF)
    if hpf:
        sos = signal.butter(5, hpf, 'highpass', output='sos')

        @scared.preprocess
        def hpfproc(traces):
            return signal.sosfilt(sos, traces)

        processes.append(hpfproc)

    # Simple low pass filter (LPF)
    if lpf:
        sos = signal.butter(5, lpf, 'lowpass', output='sos')

        @scared.preprocess
        def lpfproc(traces):
            return signal.sosfilt(sos, traces)

        processes.append(lpfproc)

    # This resync preprocessing isn't really used, was used with some JTAG captures only
    @scared.preprocess
    def resync(traces):
        import numpy as np
        first = []
        for w in traces:
            first.append(w[0])
        avg = np.mean(first)

        tracesresync = []
        for w in traces:
            if w[0] < avg:
                tracesresync.append(w[:-1])
            else:
                tracesresync.append(w[1:])
        return np.array(tracesresync)
    #processes.append(resync)

    if window:
        container = scared.Container(ths, frame=range(window[0], window[1]), preprocesses=processes)
    else:
        container = scared.Container(ths, preprocesses=processes)


    att.run(container)

    import numpy as np

    recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')

    return att

In [ ]:
# Datasets in baseline_traces_cpa online
filenames = [('baseline_traces_cpa/baseline120mhz_sync_120msps_cpa.ets', 25, None, None, None),
             ('baseline_traces_cpa/baseline120mhz_sync_60msps_cpa.ets', 25, None, None, None),
             ('baseline_traces_cpa/baseline120mhz_async_120msps_cpa.ets', 125, None, None, None),
             ('baseline_traces_cpa/baseline120mhz_async_60msps_cpa.ets', 125, (0, 3000), None, None),
             ('baseline_traces_cpa/baseline15mhz_sync_120msps_cpa.ets', 50, None, None, None),
             ('baseline_traces_cpa/baseline15mhz_sync_60msps_cpa.ets', 50, None, None, None),
             ('baseline_traces_cpa/baseline15mhz_async_120msps_cpa.ets', 50, (1000, 6000), None, None),
             ('baseline_traces_cpa/baseline15mhz_async_60msps_cpa.ets', 50, (500, 3000), None, None),
            ]


## Doing all the CPA attacks on baseline measurements

The following will do all the attacks on *baseline* measurements. The results get saved to a `.npy` file for further use. The `.npy` files have also been saved from prior runs.

The following results in the CPA numbers that are reported in the paper.

In [ ]:
for f in filenames:
    att = do_cpa(f[0], f[1], f[2], f[3], f[4])
    successnum = find_success(att, knownkey)
    print("{:s} = {:d}".format(f[0], successnum))
    np.save(f[0] + ".npy", att)

Trace Header Set:
Name.............: ETS Format THS
Reader...........: ETS format reader of file baseline_traces_cpa/baseline120mhz_sync_120msps_cpa.ets with 1000 traces.
ciphertext.......: uint8
key..............: uint8
plaintext........: uint8

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
baseline_traces_cpa/baseline120mhz_sync_120msps_cpa.ets = 750
Trace Header Set:
Name.............: ETS Format THS
Reader...........: ETS format reader of file baseline_traces_cpa/baseline120mhz_sync_60msps_cpa.ets with 4000 traces.
ciphertext.......: uint8
key..............: uint8
plaintext........: uint8

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
baseline_traces_cpa/baseline120mhz_sync_60msps_cpa.ets = 1350
Trace Header Set:
Name.............: ETS Format THS
Reader...........: ETS format reader of file baseline_traces_cpa/baseline120mhz_async_120msps_cpa.ets with 80000 traces.
ciphertext..

The above results in a bunch of output, but the important parts are the lines that look like this:

```
baseline_traces_cpa/baseline120mhz_sync_120msps_cpa.ets = 750
baseline_traces_cpa/baseline120mhz_sync_60msps_cpa.ets = 1350
baseline_traces_cpa/baseline120mhz_async_120msps_cpa.ets = 28375
baseline_traces_cpa/baseline120mhz_async_60msps_cpa.ets = 66750
baseline_traces_cpa/baseline15mhz_sync_120msps_cpa.ets = 3250
baseline_traces_cpa/baseline15mhz_sync_60msps_cpa.ets = 3600
baseline_traces_cpa/baseline15mhz_async_120msps_cpa.ets = 19700
baseline_traces_cpa/baseline15mhz_async_60msps_cpa.ets = 22800
```

These baseline values can be found in **Table 1** in the paper, reproduced here in Markdown (not exactly perfect):

| CPU Freq | Asynchronous Sampling |       |         |       | Synchronous Sampling |      |         |      |
|----------|-----------------------|-------|---------|-------|----------------------|------|---------|------|
|          | 120 MS/s              |       | 60 MS/s |       | 120 MS/S             |      | 60 MS/s |      |
|          | |TVLA|                | CPA   | |TVLA|  | CPA   | |TVLA|               | CPA  | |TVLA|  | CPA  |
| 120 MHz  | 5.0                   | 28375 | 4.2     | 66750 | 43.9                 | 750  | 28.8    | 1350 |
| 15 MHz   | 5.9                   | 19900 | 5.0     | 22800 | 24.0                 | 3250 | 25.8    | 3600 |

Note depending on the reporting internal there seems to be minor variations - here the table as 19900 for 15Mhz CPU / 120 MS/s async, but re-running it here resulted in 19700.

You may wish to save the .npy files. While we have a copy of them, for a "complete" artifact recreation you should copy the .npy files to the CPA plotting notebook.

The easiest way to do this if using colab (and not a local Jupyter install) is to copy them to a directory & compress them. Then download the file in the CoLab web interface.

The following block will move them to a `results` directory, then the block after that will compress that directory. Look for a `results.7z` file you download.

In [ ]:
%%bash
mkdir results
mv baseline_traces_cpa/*.npy results/.

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!7z a results.7z results

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive:
  0M Scan           1 folder, 8 files, 2400918836 bytes (2290 MiB)

Creating archive: results.7z

Items to compress: 9

  0%      0% + results/baseline120mhz_async_120msps_cpa.ets.npy                                                         1% + results/baseline120mhz_async_120msps_cpa.ets.npy                                                         2% + results/baseline120mhz_async_120msps_cpa.ets.npy                                                       

You will now need to reproduce the T-Test results in Table 1. See the next artifact to do this.